[자연어 전 처리]
- [1] 데이터 준비
    - 데이터 다운로드
    - 파일 데이터 프레임으로 불러오기.
- [2] 전처리
    - [2-1] 기본 전처리 : 불필요한 컬럼 삭제, 결측치 처리, 중복값 확인
    - [2-2] 텍스트 전처리 
    - [2-3] 토큰화

In [2]:
# 모듈 불러오기
import pandas as pd

In [3]:
songDF = pd.read_csv('real_real_all_genre.csv', usecols = ['title', 'artist', 'lyric', 'genre'])
#songDF

In [4]:
# 파일 확인
print(f'songDF.shape = {songDF.shape}\n')
print(f'songDF.info()\n{songDF.info()}\n')
print(f'songDF.head()\n{songDF.head()}\n')
print(f'songDF.columns : {songDF.columns}')

songDF.shape = (772, 4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 772 entries, 0 to 771
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   772 non-null    object
 1   artist  772 non-null    object
 2   lyric   772 non-null    object
 3   genre   772 non-null    object
dtypes: object(4)
memory usage: 24.2+ KB
songDF.info()
None

songDF.head()
        title         artist  \
0       Reset     선재 (snzae)   
1     Tip Toe         EUNSAN   
2    와사비가 최고야  projectsummit   
3  Black Card    summitmusic   
4   southside            이효승   

                                               lyric   genre  
0  I ain't waiting for your care\n다 귀찮아 dizzy cal...  hiphop  
1  oh baby girl I'll never let you go\n너 없인 못사는 놈...  hiphop  
2  완전 와사비 중독 나\n코에서 뻥 나오는 매콤한 향기\n혀끝에서 터지는 톡 쏘는 맛...  hiphop  
3  Yeah, we living that black card life, you know...  hiphop  
4  iam from south side\ncappers 전부 박살 (박살)\nrap m...  hiphop  

songDF.c

In [5]:
# 결측치, 중복값 확인 -> 중복값 drop
songDF.isnull().sum()
songDF.duplicated().sum()
songDF.drop_duplicates(keep = 'first', inplace=True)

print(songDF.isnull().sum(), songDF.duplicated().sum())

title     0
artist    0
lyric     0
genre     0
dtype: int64 0


In [6]:
# 피처, 레이블 분리
labelSR = songDF['genre']
featureDF = songDF.lyric.to_frame()
#print(f'labelSR => {labelSR}, lyricDF => {featureDF}')

In [7]:
# 클래스 확인
labelSR.value_counts()

genre
trot      114
fork      100
balad      97
indi       97
rb         95
rock       88
dance      87
hiphop     82
Name: count, dtype: int64

In [8]:
from nlp_hw2 import clean_data

featureDF.lyric = featureDF.lyric.str.replace('\n', ' ')
featureDF.lyric = clean_data(featureDF.lyric)

# 변경확인
#featureDF.lyric.head(10)

# 소문자로 통일
featureDF['lyric'] = featureDF['lyric'].str.lower()

토큰화 & 단어 사전화

In [9]:
from konlpy.tag import Kkma

# 인스턴스 생성
kkma = Kkma()


In [10]:
# 하나의 문자열로 연결
# result = featureDF.lyric.str.cat(featureDF['lyric'])
# result

In [11]:
vocab = {}
tokenList = []
for idx in range(featureDF.shape[0]) :
    morph = kkma.morphs(featureDF.iloc[idx][0])
    tokenList.append(morph)
    #print(type(morph))

    for word in morph :
        if len(word)>=2 : # 길이 2 이상인 것만 vocab에 추가
            if vocab.get(word) :
                vocab[word] +=1
            else :
                vocab[word] = 1

In [12]:
cnt = 0
for text in tokenList :
    print(text)
    cnt += 1
    if cnt == 10 : break

['i', 'aint', 'waiting', 'for', 'your', 'care', '다', '귀찮', '아', 'dizzy', 'calls', '일단', '잠시', '멀리', 'we', 'want', 'a', 'reset', 'every', 'pains', 'gonna', 'hoppin', 'regret', 'dizzy', 'fail', 'fuzzy', 'dreams', 'wrong', 'mistakes', 'back', 'all', 'pains', 'and', 'greeds', 'regret', 'its', 'gonna', 'rainy', 'day', 'if', 'you', 'stand', 'in', 'a', 'rain', 'spend', 'your', 'all', 'day', 'dance', 'in', 'a', 'rain', 'you', 'can', 'do', 'that', 'back', 'all', 'pains', 'and', 'greeds', 'reset', 'new', 'choice', 'oh', 'reset', 'heart', 'starts', 'run', 'reset', 'plans', 'gotta', 'work', 'this', 'time', 'reset', 'pains', 'and', 'greeds', 'regret', 'reset', 'heart', 'starts', 'run', 'we', 'want', 'a', 'reset', 'every', 'pains', 'gonna', 'hoppin', 'regret', 'dizzy', 'fail', 'fuzzy', 'dreams', 'wrong', 'mistakes', 'back', 'all', 'pains', 'and', 'greeds', 'regret', 'its', 'gonna', 'rainy', 'day', 'if', 'you', 'stand', 'in', 'a', 'rain', 'spend', 'your', 'all', 'day', 'dance', 'in', 'a', 'rain', 'yo

In [13]:
#len(vocab) #8210

In [14]:
#sorted(vocab.items(), key = lambda x : x[1], reverse = True)

In [15]:
# 불용어 처리 (2) - 영어 불용어 삭제.
import nltk
#nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords = stopwords.words('english') # 리스트 타입

# 불용어 안에 아포스트로피(') 삭제
stopwords = [word.replace("'", "") for word in stopwords]
print(stopwords)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'youre', 'youve', 'youll', 'youd', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'shes', 'her', 'hers', 'herself', 'it', 'its', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'thatll', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', '

In [16]:
clean = {token : value for token, value in vocab.items() if token not in stopwords}

sorted(clean.items(), key = lambda x : x[1], reverse = True)

[('사랑', 920),
 ('우리', 825),
 ('마음', 528),
 ('im', 481),
 ('어도', 437),
 ('어요', 388),
 ('다시', 385),
 ('에서', 372),
 ('like', 367),
 ('생각', 364),
 ('oh', 362),
 ('love', 361),
 ('처럼', 338),
 ('시간', 326),
 ('know', 322),
 ('버리', 313),
 ('사람', 313),
 ('지만', 310),
 ('어서', 308),
 ('으로', 302),
 ('ㄹ까', 292),
 ('baby', 286),
 ('그대', 283),
 ('에게', 282),
 ('오늘', 282),
 ('나의', 273),
 ('이제', 267),
 ('함께', 265),
 ('못하', 261),
 ('ㄴ다', 259),
 ('하나', 249),
 ('모르', 247),
 ('지금', 241),
 ('기억', 238),
 ('세상', 236),
 ('내가', 236),
 ('모든', 234),
 ('너무', 232),
 ('아도', 231),
 ('행복', 223),
 ('그리하', 222),
 ('는데', 220),
 ('당신', 217),
 ('말하', 207),
 ('바람', 203),
 ('아니', 201),
 ('순간', 199),
 ('yeah', 197),
 ('너와', 192),
 ('아직', 184),
 ('하루', 184),
 ('여기', 183),
 ('모두', 179),
 ('wanna', 179),
 ('want', 177),
 ('어야', 177),
 ('없이', 175),
 ('go', 174),
 ('uh', 174),
 ('같이', 170),
 ('매일', 169),
 ('cant', 169),
 ('아요', 167),
 ('만나', 162),
 ('보이', 159),
 ('그냥', 159),
 ('la', 159),
 ('time', 158),
 ('기다리', 157),
 ('잖아', 152),

In [17]:
#len(clean) #8083

In [18]:
# 불용어 제거 (3) - 한국어
filename = 'stop_word.txt'
stop_kor = set(open(filename).read().splitlines())

In [19]:
clean = {token : value for token, value in clean.items() if token not in stop_kor}
sorted(clean.items(), key = lambda x : x[1], reverse = True)

[('사랑', 920),
 ('마음', 528),
 ('im', 481),
 ('어도', 437),
 ('어요', 388),
 ('다시', 385),
 ('like', 367),
 ('생각', 364),
 ('oh', 362),
 ('love', 361),
 ('처럼', 338),
 ('know', 322),
 ('버리', 313),
 ('사람', 313),
 ('어서', 308),
 ('ㄹ까', 292),
 ('baby', 286),
 ('그대', 283),
 ('오늘', 282),
 ('나의', 273),
 ('이제', 267),
 ('못하', 261),
 ('ㄴ다', 259),
 ('모르', 247),
 ('지금', 241),
 ('기억', 238),
 ('세상', 236),
 ('내가', 236),
 ('모든', 234),
 ('너무', 232),
 ('아도', 231),
 ('행복', 223),
 ('그리하', 222),
 ('는데', 220),
 ('말하', 207),
 ('바람', 203),
 ('순간', 199),
 ('yeah', 197),
 ('너와', 192),
 ('아직', 184),
 ('하루', 184),
 ('wanna', 179),
 ('want', 177),
 ('어야', 177),
 ('없이', 175),
 ('go', 174),
 ('uh', 174),
 ('매일', 169),
 ('cant', 169),
 ('아요', 167),
 ('만나', 162),
 ('보이', 159),
 ('그냥', 159),
 ('la', 159),
 ('time', 158),
 ('기다리', 157),
 ('잖아', 152),
 ('아서', 151),
 ('ㄹ래', 149),
 ('say', 149),
 ('one', 148),
 ('그렇', 148),
 ('아프', 147),
 ('보다', 146),
 ('beep', 144),
 ('으면', 142),
 ('night', 141),
 ('got', 139),
 ('bad', 138),
 ('정

In [20]:
#clean.get('어')

In [21]:
# 불용어 [3] - 어말어미 및 연결어미 삭제
grammar = ['ㄴ다', '다', 'ㄴ지', '은지', 'ㄴ다는', '는다는', '라는', '이나', '나', '이라는', '라서', '이라서', '니까', '으니까', '고', 'ㄹ지', 'ㄹ라', '을라', '을지', '구나', 'ㄹ걸', '을걸', 'ㄹ래', '을래요', 'ㄹ래요', 'ㄹ게', '을게', 'ㄹ까', '을까', 'ㅂ니다', '습니다', '읍시다', 'ㅂ시다', '든지', '어서', '어야', '어요', '어야지', '어도', '아도', '으며', '으면서', '면서', '라도', '이라도', '아요']
cleanDict = {token : value for token, value in clean.items() if token not in grammar}

In [22]:
#불용어 [4] - 영어 
stop_eng = ['im','cant']
cleanDict = {token : value for token, value in cleanDict.items() if token not in stop_eng}

In [23]:
len(clean) #> 7914

7952

In [24]:
#
cleanList = sorted(clean.items(), key = lambda x : x[1], reverse = True)
# wordDict = {0:'<pad>', 1:'<unk>'}
# for idx, (word, _) in enumerate(clean.item()) : 
#     wordDict[idx+2] = word

wordList = ['<pad>', '<unk>']
for (word, _) in cleanList :
    wordList.append(word)

In [27]:
#pd.DataFrame(wordList).to_csv('vocab.csv')
#print('csv변환')

csv변환


인코딩 인덱싱

In [ ]:
# 인코딩 : 문자 -> 숫자
encode = {token : idx for idx, token in enumerate(wordList)}
print(encode)

{'<pad>': 0, '<unk>': 1, '사랑': 2, '마음': 3, 'im': 4, '어도': 5, '어요': 6, '다시': 7, 'like': 8, '생각': 9, 'oh': 10, 'love': 11, '처럼': 12, 'know': 13, '버리': 14, '사람': 15, '어서': 16, 'ㄹ까': 17, 'baby': 18, '그대': 19, '오늘': 20, '나의': 21, '이제': 22, '못하': 23, 'ㄴ다': 24, '모르': 25, '지금': 26, '기억': 27, '세상': 28, '내가': 29, '모든': 30, '너무': 31, '아도': 32, '행복': 33, '그리하': 34, '는데': 35, '말하': 36, '바람': 37, '순간': 38, 'yeah': 39, '너와': 40, '아직': 41, '하루': 42, 'wanna': 43, 'want': 44, '어야': 45, '없이': 46, 'go': 47, 'uh': 48, '매일': 49, 'cant': 50, '아요': 51, '만나': 52, '보이': 53, '그냥': 54, 'la': 55, 'time': 56, '기다리': 57, '잖아': 58, '아서': 59, 'ㄹ래': 60, 'say': 61, 'one': 62, '그렇': 63, '아프': 64, '보다': 65, 'beep': 66, '으면': 67, 'night': 68, 'got': 69, 'bad': 70, '정말': 71, '눈물': 72, 'ㄴ데': 73, 'feel': 74, '을까': 75, 'ㄹ게': 76, '가슴': 77, '이야': 78, '하늘': 79, 'let': 80, '인생': 81, 'get': 82, '그러': 83, '모습': 84, '이렇': 85, '내리': 86, '노래': 87, 'ㄹ지': 88, '원하': 89, '바라보': 90, 'heart': 91, 'ill': 92, '해지': 93, '느끼': 94, '그리': 95, '아무'

In [ ]:
# 리뷰의 문자를 정수로 변환 / 단어사전에 없는 문자 처리

UNK_ID = encode.get('<unk>')
id = [[encode.get(token, UNK_ID) for token in text] for text in tokenList]    
print(id)
print(len(id))
#print(UNK_ID)
# id = [encode.get(token, UNK_ID) for token in morph]


[[1, 396, 487, 1, 1, 350, 1, 841, 1, 2407, 2934, 1, 1, 145, 1, 44, 1, 1247, 98, 1494, 109, 3621, 1659, 2407, 2124, 3622, 418, 1055, 2935, 121, 1, 1494, 1, 1852, 1659, 1, 109, 2125, 127, 1, 1, 504, 1, 1, 637, 2408, 1, 1, 127, 223, 1, 1, 637, 1, 1, 1, 1, 121, 1, 1494, 1, 1852, 1247, 320, 2936, 10, 1247, 91, 3623, 118, 1247, 5148, 842, 1853, 1, 56, 1247, 1494, 1, 1852, 1659, 1247, 91, 3623, 118, 1, 44, 1, 1247, 98, 1494, 109, 3621, 1659, 2407, 2124, 3622, 418, 1055, 2935, 121, 1, 1494, 1, 1852, 1659, 1, 109, 2125, 127, 1, 1, 504, 1, 1, 637, 2408, 1, 1, 127, 223, 1, 1, 637, 1, 1, 1, 1, 121, 1, 1494, 1, 1852, 1659, 1, 116, 1, 1150, 1494, 1, 1852, 1659, 1], [10, 18, 130, 92, 119, 80, 1, 47, 1, 1, 46, 1, 2937, 1, 1, 1, 2126, 1, 366, 997, 1, 1, 536, 1, 843, 83, 1, 1, 1, 1248, 1, 1, 1, 1, 1, 1, 261, 1, 638, 382, 1, 1, 1, 1, 1, 1, 1, 1, 419, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 801, 1, 5149, 10, 1, 1, 1, 1, 1, 1, 488, 639, 1, 40, 1, 1, 1, 1, 2408, 1, 998, 84, 1, 505, 1, 1, 1, 1, 1, 1, 447, 1, 1, 1, 85,

In [ ]:
# 패딩 처리
# 가장 긴 문장의 길이 찾기.
max_length = 0
for i in id :
    if max_length<len(i) :
        max_length = len(i)

print(max_length)

1695


In [ ]:
# 학습용, 테스트용 데이터 패딩 처리
padded_id=[]
for i in id :
    if len(i) < max_length :
            padded_id.append(i + [0]*(max_length-len(i)))
    else:
        padded_id.append(i)

In [ ]:
padded_id[4]

[1667,
 1,
 1668,
 474,
 1857,
 1,
 1059,
 1059,
 1060,
 671,
 1858,
 1,
 1,
 16,
 563,
 152,
 1000,
 152,
 1859,
 244,
 39,
 1667,
 1,
 1668,
 474,
 1857,
 1,
 1059,
 1059,
 1060,
 671,
 1858,
 1,
 1,
 16,
 563,
 152,
 1000,
 152,
 1859,
 244,
 39,
 1,
 8,
 1,
 1,
 1,
 5173,
 29,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 39,
 5174,
 262,
 16,
 1157,
 1,
 1,
 5175,
 1,
 1,
 2955,
 2956,
 1,
 2414,
 1,
 1,
 1,
 2957,
 1,
 1,
 1,
 73,
 5176,
 1,
 1,
 1,
 5177,
 940,
 1,
 23,
 1,
 2128,
 2958,
 3638,
 41,
 263,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 5178,
 1,
 1,
 2957,
 334,
 5179,
 1371,
 1,
 5180,
 1,
 1,
 5181,
 1,
 5182,
 5183,
 1,
 3639,
 2129,
 3639,
 716,
 1372,
 1,
 2130,
 3640,
 1,
 1158,
 53,
 1,
 1373,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 14,
 60,
 5184,
 1860,
 1,
 590,
 1,
 5185,
 1,
 1,
 1,
 89,
 3641,
 5186,
 1,
 5187,
 1252,
 3642,
 1,
 1669,
 334,
 1,
 1,
 1,
 2959,
 3643,
 1253,
 2415,
 1,
 1,
 1,
 1,
 1,
 3644,
 2960,
 12,
 87,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1667,
 1,
 1668,
 474,
 1857,
 1,
 1059,
 1

In [ ]:
# encodingDict = {}
# for idx in range(len(id)) :
#     encodingDict[idx] = id[idx]
#print(encodingDict[168])
#print(encodingDict)>> 터짐

데이터 가공
- 데이터 길이 표준화
    - 토큰 정수화
    - 데이터 구성 단어 수 맞추기(패딩)

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelEncoder= LabelEncoder()
encoded_labels = labelEncoder.fit_transform(labelSR)
#encoded_labels

In [ ]:
# df = pd.DataFrame({'encoded_lyrics': encodingDict.values(), 'encoded_labels':encoded_labels})
index = list(range(0,760))
df = pd.DataFrame(padded_id, index=index)
df['genre'] = encoded_labels
df

,0,1,2,3,4,5,6,7,8,9,...,1686,1687,1688,1689,1690,1691,1692,1693,1694,genre
0,1,396,487,1,1,350,1,841,1,2407,...,0,0,0,0,0,0,0,0,0,3
1,10,18,130,92,119,80,1,47,1,1,...,0,0,0,0,0,0,0,0,0,3
2,1370,1154,1661,1,1,1,1,1,140,1,...,0,0,0,0,0,0,0,0,0,3
3,39,1,1057,1,317,1249,176,1,13,3633,...,0,0,0,0,0,0,0,0,0,3
4,1667,1,1668,474,1857,1,1059,1059,1060,671,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,1,1,1,1,1,75,1,1,1237,1,...,0,0,0,0,0,0,0,0,0,2
756,3620,1,140,1,7938,1,318,1,1,1,...,0,0,0,0,0,0,0,0,0,2
757,1,1,1,919,1,1,234,1,94,202,...,0,0,0,0,0,0,0,0,0,2
758,28,1,1,1,1,1,20,42,1,256,...,0,0,0,0,0,0,0,0,0,2


In [ ]:
# df.to_csv('encoded.csv')
# print('csv 파일로 저장 완료')

csv 파일로 저장 완료
